In [1]:
import mne
import os
# os.chdir('C:\\Users\\Esimk_000\\Desktop\\School\\Spring 2018\\Cs59866\\EEG Project\\seniorprojecteeg-master-fork\\preprocessing\\Artifact_Removal')
%run preprocecssing_helpers.ipynb
%run Batch_ArtifactFilter_Epoch.ipynb
import time
import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 15.0)
matplotlib.rcParams.update({'font.size': 15})

In [11]:
# path = 'D:/EEGLAB/'
# eeglab_files = list_eeglab_files(path)

In [12]:
# eeglab_files

['D:/EEGLAB/SA20131216\\20131216_1310.set',
 'D:/EEGLAB/SA20131216\\20131216_1441.set',
 'D:/EEGLAB/SA20140205\\20140205_1114.set',
 'D:/EEGLAB/SA20140205\\20140205_1230.set',
 'D:/EEGLAB/SA20140206\\20140206_1029.set',
 'D:/EEGLAB/SA20140206\\20140206_1151.set',
 'D:/EEGLAB/SA20140207\\20140207_1146.set',
 'D:/EEGLAB/SA20140207\\20140207_1219.set',
 'D:/EEGLAB/SA20140217\\20140217_1136.set',
 'D:/EEGLAB/SA20140217\\20140217_1211.set',
 'D:/EEGLAB/SA20140219\\20140219_1046.set',
 'D:/EEGLAB/SA20140219\\20140219_1205.set',
 'D:/EEGLAB/SA20140220\\20140220_1009.set',
 'D:/EEGLAB/SA20140220\\20140220_1123.set',
 'D:/EEGLAB/SA20140225\\20140225_1422.set',
 'D:/EEGLAB/SA20140225\\20140225_1552.set',
 'D:/EEGLAB/SA20140227A\\20140227_1058.set',
 'D:/EEGLAB/SA20140227A\\20140227_1220.set',
 'D:/EEGLAB/SA20140227B\\20140227_1410.set']

In [13]:
# raw = mne.io.read_raw_eeglab(eeglab_files[4],preload = True)

Reading D:/EEGLAB/SA20140206\20140206_1029.fdt
Reading 0 ... 813578  =      0.000 ...  1589.020 secs...


In [ ]:
# os.chdir("F:\\")
# epoch.save(eeglab_files[4].split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)

In [14]:
# events = find_events(raw)
# eeglab_trials = get_trials(events)
# eeglab_trials[1:3]

5792 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]


[[array([7983,    0,   31], dtype=int64),
  array([7986,    0,  136], dtype=int64),
  array([7989,    0,  156], dtype=int64),
  array([8000,    0,   11], dtype=int64),
  array([8276,    0,   12], dtype=int64),
  array([8526,    0,   10], dtype=int64),
  array([9077,    0,  132], dtype=int64),
  array([9080,    0,  133], dtype=int64),
  array([9082,    0,  129], dtype=int64)],
 [array([9378,    0,   31], dtype=int64),
  array([9381,    0,  143], dtype=int64),
  array([9384,    0,  156], dtype=int64),
  array([9396,    0,   11], dtype=int64),
  array([9671,    0,   12], dtype=int64),
  array([9930,    0,   10], dtype=int64),
  array([10175,     0,   131], dtype=int64),
  array([10197,     0,   134], dtype=int64),
  array([10199,     0,   129], dtype=int64)]]

In [7]:
def get_epoch_trials(epoch_object):
    #note: third stim-code has a value that is 1 more than the first.
    events = epoch_object.events
    found_first_stim_code = False
    found_second_stim_code = False
    found_third_stim_code = False
    current_trial = []
    trials = []
    for event in events:
        if not found_first_stim_code and not found_second_stim_code and not found_third_stim_code: #still searching for the end of trial
            current_trial.append(event)
            found_first_stim_code = True
            first_stim_code = event
        elif not found_second_stim_code and not found_third_stim_code:
            current_trial.append(event)
            found_second_stim_code = True
        elif not found_third_stim_code and (event[-1] + 1) == first_stim_code[-1]: 
            current_trial.append(event)
            found_third_stim_code = True
            trials.append(current_trial)
            current_trial = []
            found_first_stim_code = False
            found_second_stim_code = False
            found_third_stim_code = False
        elif not found_third_stim_code and (event[-1] + 1) != first_stim_code[-1]:
            # there is no third event, trial is unfinished.
            current_trial = []
            current_trial.append(event)#assuming it's first stim-code...for now
            found_first_stim_code = True
    return trials

In [8]:
def isCorrect_stim_code_trials(trials):
    """
    Description:
        A sanity check that makes sure all trials are in the correct order
        The third stim-code needs to have a value that is one less than the 
        first stim-code.
        
    Variables:
        trials: An ndarray of shape  (n_trials, 3, 3)
    -----
    returns:
        boolean if all are correct, or one is incorrect. 
        It will print all incorrect trials
    """
    has_incorrect_trial = True
    for trial in trials:
        if trial[:,-1][0] == trial[:,-1][-1] + 1:
            pass
        else:
            print("Incorrect Trial \n: {0}".format(trial))
            has_incorrect_trial = False
            
                    
    return has_incorrect_trial

In [ ]:
# isCorrect_stim_code_trials(epoch_trials)

In [71]:
# epoch_trials.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# def create_describer_events(epoch, eeglab):
#     """
#     Description:
#         uses raw eeglab events and epoch events to return a new dictionary
#         with key the timestamp of an event, and value is the new event_id.
#         These event_ids can later be split/manipulated to label data by specific 
#         events
#     Variables:
#         epoch: mne.io.epoch object
#         raw_events: an ndarray of 3d integers shape 
        
#     returns:
#         dictionary
#     """
#     raw_events = find_events(eeglab)
#     raw_trials = get_trials(raw_events)
#     epoch_events = epoch.events
#     epoch_trials = get_epoch_trials(epoch)
    
#     for epoch_trial in epoch_trials:
#         for event in epoch_trial
#             if event[0] in 
#     #  

In [ ]:
# epoch_trials = np.array(get_epoch_trials(epoch))

In [3]:
# root_path = 'D:\\'

In [9]:
# l_epoch = mne.read_epochs(os.path.join(root_path,'20140206_1029_epo.fif'), preload= True)

Reading D:\20140206_1029_epo.fif ...
    Found the data of interest:
        t =       0.00 ...    1949.22 ms
        0 CTF compensation matrices available
949 matching events found
Applying baseline correction (mode: mean)
949 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [10]:
# l_epoch.events

array([[  8000,      0,     11],
       [  8276,      0,     12],
       [  8526,      0,     10],
       ...,
       [405221,      0,     12],
       [405505,      0,      7],
       [406065,      0,      8]])

In [15]:
# events = find_events(raw)
# eeglab_trials = get_trials(events)
# epoch_trials = get_epoch_trials(l_epoch)

5792 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]


In [17]:
# epoch_trials[:1]

[[array([8000,    0,   11]),
  array([8276,    0,   12]),
  array([8526,    0,   10])]]

In [23]:
# np.array(eeglab_trials).shape

(641,)

In [25]:
# np.array(epoch_trials).shape

(316, 3, 3)

In [64]:
# epoch_trials[0][0]

array([8000,    0,   11])

In [27]:
# eeglab_trials

[[array([7671,    0,  131], dtype=int64)],
 [array([7983,    0,   31], dtype=int64),
  array([7986,    0,  136], dtype=int64),
  array([7989,    0,  156], dtype=int64),
  array([8000,    0,   11], dtype=int64),
  array([8276,    0,   12], dtype=int64),
  array([8526,    0,   10], dtype=int64),
  array([9077,    0,  132], dtype=int64),
  array([9080,    0,  133], dtype=int64),
  array([9082,    0,  129], dtype=int64)],
 [array([9378,    0,   31], dtype=int64),
  array([9381,    0,  143], dtype=int64),
  array([9384,    0,  156], dtype=int64),
  array([9396,    0,   11], dtype=int64),
  array([9671,    0,   12], dtype=int64),
  array([9930,    0,   10], dtype=int64),
  array([10175,     0,   131], dtype=int64),
  array([10197,     0,   134], dtype=int64),
  array([10199,     0,   129], dtype=int64)],
 [array([10481,     0,    31], dtype=int64),
  array([10484,     0,   143], dtype=int64),
  array([10487,     0,   156], dtype=int64),
  array([10498,     0,    11], dtype=int64),
  array([1